### Importing libraries

In [3]:
from azure.cosmos import CosmosClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.cosmos.partition_key import PartitionKey

from dotenv import load_dotenv
import pandas as pd
load_dotenv(override=True)
import json
import pyodbc

import os

# Using DefaultAzureCredential (recommended)
# https://techcommunity.microsoft.com/t5/azure-architecture-blog/configure-rbac-for-cosmos-db-with-managed-identity-instead-of/ba-p/3056638#:~:text=Create%20custom%20roles%20MyReadOnlyRole%20and%20MyReadWriteRole%20with%20both,definition%20create%20-a%20%24accountName%20-g%20%24resourceGroupName%20-b%20%40role-definition-ro.json
aad_credentials = DefaultAzureCredential()

AZURE_COSMOS_DB_ENDPOINT=os.environ['AZURE_COSMOS_DB_ENDPOINT']
AZURE_COSMOS_DB_KEY= os.environ['AZURE_COSMOS_DB_KEY']
AZURE_COSMOS_DB_DATABASE= os.environ['AZURE_COSMOS_DB_DATABASE']
AZURE_COSMOS_DB_CONN= os.environ['AZURE_COSMOS_DB_CONN']
azurecosmosdbclient = CosmosClient(AZURE_COSMOS_DB_ENDPOINT, credential=aad_credentials)

CONTAINER_ID = os.environ['AZURE_COSMOS_DB_CONTAINER']
PartitionKeyPath = "/chunk_id"

database_client = azurecosmosdbclient.get_database_client(AZURE_COSMOS_DB_DATABASE)
container_client = database_client.get_container_client(CONTAINER_ID)

SQL_SERVER_USERNAME = os.environ["SQL_SERVER_USERNAME"]
SQL_SERVER_ENDPOINT = os.environ["SQL_SERVER_ENDPOINT"]
SQL_SERVER_PASSWORD = os.environ["SQL_SERVER_PASSWORD"]
SQL_SERVER_DATABASE = os.environ["SQL_SERVER_DATABASE"] 
driver = "{ODBC Driver 18 for SQL Server}"
sqlalchemy_driver = "ODBC Driver 18 for SQL Server"

## Creatng a connection to the SQL Server
conn = pyodbc.connect(f'DRIVER={driver};SERVER={SQL_SERVER_ENDPOINT};PORT=1433;DATABASE={SQL_SERVER_DATABASE};UID={SQL_SERVER_USERNAME};PWD={SQL_SERVER_PASSWORD}')  
cursor = conn.cursor()


from openai import AzureOpenAI
aoai_client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-07-01-preview"
)

### AzureSQLDB Tool calling

#### SQLDatabase GetSQLDBAnswer Function definition

In [9]:
def GetSQLDBAnswer(question):

    prompt_template = """ You are a SQL programmer Assistant.Your role is to generate SQL code (SQL Server) to retrieve an answer to a natural language query. Make sure to disambiguate column names when creating queries that use more than one table. If a valid SQL query cannot be generated, only say "ERROR:" followed by why it cannot be generated.
                  Do not answer any questions on inserting or deleting rows from the table. Instead, say "ERROR: I am not authorized to make changes to the data"

                  Use the following sales database schema to write SQL queries:
                  Customers(cust_id INTEGER, cust_name VARCHAR, cust_email VARCHAR, cust_phone VARCHAR, cust_address VARCHAR, PRIMARY KEY (cust_id))
                  Products(prod_id INTEGER,prod_name varchar, price FLOAT, category VARCHAR, PRIMARY KEY(prod_id))
                  Stock(prod_id INTEGER, merchant_id INTEGER, stock INTEGER, PRIMARY KEY(prod_id, merchant_id), FOREIGN KEY(merchant_id, prod_id))
                  Merchants(merchant_id INTEGER, merchant_name VARCHAR, merchant_region VARCHAR, merchant_address VARCHAR, PRIMARY KEY(merchant_id))
                  Sales(sale_id INTEGER, cust_id INTEGER , merchant_id INTEGER , date TIMESTAMP, total_price FLOAT, PRIMARY KEY(sale_id),FOREIGN KEY(cust_id,merchant_id))
                  Sales_Detail(sales_id INTEGER, prod_id INTEGER, quantity INTEGER, PRIMARY KEY(sales_id,prod_id), FOREIGN KEY(sales_id,prod_id))

                  ## Additional guidance
                    ** Please only answer questions that can be answered with the data provided. Do not make any assumptions about the data.
                    ** Please write the code for a SQL Server database.
                    ** If you need to ask any clafication, please ask for it in the format JSON format with the key: CLARIFICATION
                    ** If asked about any date, default to filing_date unless the question implies that is a question about the filing period.
                    ** Dates should be explicity stated in the query. For example, if you are asked about filings in 2023, you should use the following format: '2023-01-01' and '2023-12-31'. Otehrwise, ask for the specific period.
                    ** Provide the answer in a JSON format with the key: SQLquery
                  question: """
    
    messages = [{"role":"system","content":prompt_template}, 
               {"role":"user","content":question}]

    response = aoai_client.chat.completions.create(model="gpt4o",  
                                        messages = messages, 
                                        temperature=0.1,  
                                        max_tokens=2000,
                                        response_format={ "type": "json_object" },
                                        seed = 42)
    SQLschema = response.choices[0].message.content
    
    if "ERROR" in SQLschema:
        print('ERROR')
        return str({"SQLquery":"No query", "Answer": str(json.loads(SQLschema)['ERROR'])})
    elif 'CLARIFICATION' in SQLschema:
        print('CLARIFICATION')
        return str({"SQLquery":"No query", "Answer": str(json.loads(SQLschema)['CLARIFICATION'])})
    else:
        SQLQuery = json.loads(SQLschema)['SQLquery']
        print(SQLQuery)
        return str({"SQLquery":SQLQuery, "Answer": str([i for i in container_client.query_items(query=SQLQuery, enable_cross_partition_query=True)])})

In [11]:
output = GetSQLDBAnswer("how many surface devices were sold on 2023?")
print(output)

CLARIFICATION
{'SQLquery': 'No query', 'Answer': "Could you please specify the product name or category for 'surface devices'?"}


In [16]:
def run_conversation(question):
    # Initial user message
    messages = [{"role":"system","content":"You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls"}, {"role": "user", "content": question}] # Single function call

    # Define the function for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "GetCosmosDBAnswer",
                "description": """Take a query, run a NoSQL query on the CosmosDB database, and return the result. 
                Only answer questions from the following database schema: DI_Text_HTML_PageSplitter(page_num INTEGER, content VARCHAR, title VARCHAR, chunk_id VARCHAR, preprocessing_pipeline VARCHAR, filename VARCHAR, filing_period VARCHAR, filing_date VARCHAR, form_type VARCHAR, ticker VARCHAR PRIMARY KEY (chunk_id))
                This database contains chunks (sections) that were extracting from SEC filings. Mentioning company names or tickers should not trigger this function unless they ask something about the schema. An example of a question that cannot be answered by this function is: How many stocks did microsoft repurchased in 2023?
                This function should only be used to answer database-like questions.
                DO NOT use this function if the question is not related to the database schema provided.""",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "question that needs to be answered using the CosmosDB database. Please pass in verbatim the question that the user asked.",
                        },
                    },
                    "required": ["query"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the function
    response = aoai_client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        seed=42
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)


    print("Model's response:")  
    print(response_message.tool_calls)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}") 

            if function_name == "GetCosmosDBAnswer":
                function_response = GetCosmosDBAnswer(
                    question=function_args.get("question")
                )
                print(f"Function response: {function_response}")
            else:
                function_response = "Function not found"
            
        messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("No tool calls were made by the model.")
        messages.append({"role": "system", "content": "No functions or tools were called by the model."})
    
    
    print(messages)
    final_response = aoai_client.chat.completions.create(
        model='gpt4o',
        messages=messages,
        seed=42
    )

    return final_response.choices[0].message.content

In [17]:
run_conversation("Who many quarterly filings were done in the last 2 quarters of 2023?")

Model's response:
[ChatCompletionMessageToolCall(id='call_Tcz75D8T7YGJHQqJyphx3sFq', function=Function(arguments='{"question":"How many quarterly filings were done in the last 2 quarters of 2023?"}', name='GetCosmosDBAnswer'), type='function')]
Function call: GetCosmosDBAnswer
Function arguments: {'question': 'How many quarterly filings were done in the last 2 quarters of 2023?'}
SELECT VALUE COUNT(1) FROM (SELECT DISTINCT c.filename FROM c WHERE c.form_type = '10Q' AND (c.filing_date BETWEEN '2023-07-01' AND '2023-12-31'))
Function response: {'NoSQLQuery': "SELECT VALUE COUNT(1) FROM (SELECT DISTINCT c.filename FROM c WHERE c.form_type = '10Q' AND (c.filing_date BETWEEN '2023-07-01' AND '2023-12-31'))", 'Answer': '[5]'}
[{'role': 'system', 'content': 'You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the s

'There were 5 quarterly filings done in the last 2 quarters of 2023.'

In [18]:
run_conversation("Who won the world cup in 2023?")

Model's response:
None
No tool calls were made by the model.
[{'role': 'system', 'content': 'You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls'}, {'role': 'user', 'content': 'Who won the world cup in 2023?'}, ChatCompletionMessage(content="I don't have access to the source of that information.", role='assistant', function_call=None, tool_calls=None), {'role': 'system', 'content': 'No functions or tools were called by the model.'}]


"I don't have access to the source of that information."